In [11]:
!pip install wget | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1


In [107]:
url = "https://us-south.ml.cloud.ibm.com"
apikey='hNMEbzdJO48ETdssoUHxArx9yPYe54aXdY3RiX0UvMW8'


credentials = {
    "url": url,
    "apikey": apikey
}



In [108]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")


In [109]:
project_id


'aba42628-cabb-4f56-82a8-ef7c75e40c13'

In [110]:
import  types
import pandas as pd
from botocore.client import Config
import ibm_boto3

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=' hNMEbzdJO48ETdssoUHxArx9yPYe54aXdY3RiX0UvMW8',  
    ibm_auth_endpoint='',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)


In [111]:
bucket = 'bucket-a4yq2mfh9r0fxkc'
object_key = 'ecommerce_reviews_train.csv'


In [115]:
body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

if not hasattr(body, "_iter"): body.iter_ = types.MethodType(iter, body)

train_data = pd.read_csv(body)
train_data.head(5)

,ReviewID,Feedback,Sentiment,Product_Category
0,1001,This vacuum cleaner picks up dirt really well.,0,Home Appliances
1,1002,Laptop runs smoothly but gets warm under heavy...,1,Electronics
2,1003,Headphones are comfortable and have crisp sound.,1,Electronics
3,1004,Amazing camera with high resolution photos!,1,Electronics
4,1005,Dishwasher cleans thoroughly but doesn't dry d...,0,Home Appliances


In [113]:
lable_map={
     0:'negative',
     1: 'positive'
}
train_data.value_counts(['Sentiment'])


Sentiment
0            576
1            424
Name: count, dtype: int64

In [114]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def iter(self): yield from self
    
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='hNMEbzdJO48ETdssoUHxArx9yPYe54aXdY3RiX0UvMW8',
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'bucket-a4yq2mfh9r0fxkc'
object_key = 'ecommerce_reviews_train.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

if not hasattr(body, "_iter"): body.iter_ = types.MethodType(iter, body)

test_data = pd.read_csv(body)
test_data.head(5)

,ReviewID,Feedback,Sentiment,Product_Category
0,1001,This vacuum cleaner picks up dirt really well.,0,Home Appliances
1,1002,Laptop runs smoothly but gets warm under heavy...,1,Electronics
2,1003,Headphones are comfortable and have crisp sound.,1,Electronics
3,1004,Amazing camera with high resolution photos!,1,Electronics
4,1005,Dishwasher cleans thoroughly but doesn't dry d...,0,Home Appliances


In [93]:
test_data.value_counts(['Sentiment'])

Sentiment
0            170
1            130
Name: count, dtype: int64

In [94]:
train_data.shape

(1000, 10)

In [95]:
test_data.shape

(300, 10)

In [116]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])


model_id = ModelTypes.FLAN_T5_XXL

from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY: 1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1
}



['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


In [117]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [118]:
import time

# 1) Make sure this is your only “instruction” variable
instruction = """Determine the sentiment of the sentence.
Use either 'positive' or 'negative'. Use the provided examples as a template."""

# 2) Grab your feedback list
feedback_list = train_data['Feedback'].tolist()

results = []
# 3) Loop over just the first 5 items to debug
for i, text in enumerate(feedback_list[:5]):
    # Build the prompt
    prompt = f"{instruction}\n\nsentence:\t{text}\nSentiment:"
    
    # Print progress so you know it’s running
    print(f"\n---- Example {i+1} ----")
    print(prompt)
    
    # Call the model
    out = model.generate(prompt)
    pred = out["results"][0]["generated_text"].strip()
    print("Model says:", repr(pred))
    
    results.append(pred)
    
    # Sleep to avoid rate‑limit
    time.sleep(0.6)

print("\nDone. Collected preds:", results)



---- Example 1 ----
Determine the sentiment of the sentence.
Use either 'positive' or 'negative'. Use the provided examples as a template.

sentence:	This vacuum cleaner picks up dirt really well.
Sentiment:
Model says: 'positive'

---- Example 2 ----
Determine the sentiment of the sentence.
Use either 'positive' or 'negative'. Use the provided examples as a template.

sentence:	Laptop runs smoothly but gets warm under heavy load.
Sentiment:
Model says: 'positive'

---- Example 3 ----
Determine the sentiment of the sentence.
Use either 'positive' or 'negative'. Use the provided examples as a template.

sentence:	Headphones are comfortable and have crisp sound.
Sentiment:
Model says: 'positive'

---- Example 4 ----
Determine the sentiment of the sentence.
Use either 'positive' or 'negative'. Use the provided examples as a template.

sentence:	Amazing camera with high resolution photos!
Sentiment:
Model says: 'positive'

---- Example 5 ----
Determine the sentiment of the sentence.
Use e

In [119]:
zero_shot_inputs = [{"input": text} for text in test_data['Feedback']]
for i in range(2):
    print(f"The sentence example {i+1} is:\n\t {zero_shot_inputs[i]['input']}\n")



The sentence example 1 is:
	 This vacuum cleaner picks up dirt really well.

The sentence example 2 is:
	 Laptop runs smoothly but gets warm under heavy load.



In [120]:
data_train = train_data.copy()
data_test = test_data.copy()


In [121]:
few_shot_example = []

# Correct and safe sampling by preserving column names
for _, row in data_train.groupby('Sentiment').apply(lambda x: x.sample(2)).iterrows():
    phrase = row['Feedback']  # or 'Phrase' if that's the column name
    sentiment = row['Sentiment']
    few_shot_example.append(f"\tsentence:\t{phrase}\n\tSentiment: {sentiment}\n")

few_shot_examples = '\n'.join(few_shot_example)


In [122]:
few_shot_inputs_ = [{"input": text} for text in test_data['Feedback'].values]


In [123]:
test_data['Sentiment'].values


array([0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [124]:
print(data_test[['Feedback', 'Sentiment']].head(2))



                                            Feedback  Sentiment
0     This vacuum cleaner picks up dirt really well.          0
1  Laptop runs smoothly but gets warm under heavy...          1


In [125]:
# Safer and clearer approach:
few_shot_inputs_ = [{"input": row["Feedback"]} for _, row in data_test.iterrows()]
y_test = data_test["Sentiment"].reset_index(drop=True)  # Ensures index matches few_shot_inputs_

for i in range(2):
    print(f"The sentence example {i+1} is:\n {few_shot_inputs_[i]['input']}\n")
    print(f"\tSentiment: {y_test.iloc[i]}\n")


The sentence example 1 is:
 This vacuum cleaner picks up dirt really well.

	Sentiment: 0

The sentence example 2 is:
 Laptop runs smoothly but gets warm under heavy load.

	Sentiment: 1



In [ ]:
results = []
for inp in few_shot_inputs_[:2]:
    # Build the prompt
    prompt = instruction + few_shot_examples + f"\nsentence:\t{inp['input']}\n\tSentiment:"
    
    # Generate model output
    output = model.generate(prompt)
    results.append(output["results"][0])  # append the dict with 'generated_text' key

# Print raw output
import json
print(json.dumps(results, indent=2))

# Label mapping
label_map = {
    "positive": 1,
    "negative": 0
}

# Extract predictions from generated text
y_pred = [label_map[result['generated_text'].strip().lower()] for result in results]

# Ground-truth values
y_true = y_test.values[:2].tolist()

print("y_true:", y_true)
print("y_pred:", y_pred)
